# Problem Set 2
## Zhaoyang Chen 

In [29]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask import compute, delayed, multiprocessing

import time
import random
import warnings
warnings.filterwarnings("ignore")

## Question 1

In [19]:
data = pd.read_csv('./data/Auto.csv')

In [20]:
def check_mpg(mpg, median):
    if mpg < median:
        return 0
    else:
        return 1
    
def check_orgn_1(origin):
    if origin == 1:
        return 1
    else:
        return 0
def check_orgn_2(origin):
    if origin == 2:
        return 1
    else:
        return 0

data['mpg_high'] = data['mpg'].apply(check_mpg, args = (np.median(data['mpg']),))
data['orgn1'] = data['origin'].apply(check_orgn_1)
data['orgn2'] = data['origin'].apply(check_orgn_2)
data['horsepower'][data['horsepower'] == '?'] = 100

data = data[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year','orgn1','orgn2','mpg_high']]

In [21]:
def boot_LR(data):
    train = data.sample(frac = 0.65, replace = True)
    test = data.drop(train.index, axis = 0)
    
    X_train = train.drop('mpg_high', axis = 1)
    y_train = train['mpg_high']
    X_test = test.drop('mpg_high', axis = 1)
    y_test = test['mpg_high']
    
    model = LogisticRegression().fit(X_train, y_train)
    error = 1 - model.score(X_test, y_test)
    
    return error
random.seed(1234)
error_rate = []
start_time = time.time()
for i in range(100):
    error_rate.append(boot_LR(data))
print('The average error rate is {}%.'.format(np.mean(error_rate)*100))
print('The running time is {}s.'.format(time.time()-start_time))

The average error rate is 10.115926873670901%.
The running time is 0.4440279006958008s.


In [22]:
client = Client()
client.cluster

In [35]:
start_time = time.time()
def parallel_boot_LR(boot_num, seed, data):
    random.seed(seed)
#     print('Working on Bootstrap', boot_num)
    
    train = data.sample(frac = 0.65, replace = True)
    test = data.drop(train.index, axis = 0)
    
    X_train = train.drop('mpg_high', axis = 1)
    y_train = train['mpg_high']
    X_test = test.drop('mpg_high', axis = 1)
    y_test = test['mpg_high']
    
    model = LogisticRegression().fit(X_train, y_train)
    error = 1 - model.score(X_test, y_test)
    
    return error

lazy_values = []
for i in range(100):
    lazy_values.append(delayed(parallel_boot_LR)(i, 1234, data))
result = compute(*lazy_values, scheduler=multiprocessing.get)
    

print('The average error rate is {}%.'.format((parallel_boot_LR(100,1234,data)*100)))
print('The running time is {}s'.format(time.time()-start_time))
    

The average error rate is 12.918660287081341%.
The running time is 0.49239301681518555s
